Imports:

In [1]:
import numpy as np
import pandas as pd  
import data_cleaning_for_EDA as dc 
import matplotlib.pyplot as plt
import seaborn as sns
import imputing_methods as im
# from imputing_methods import Plotter, DataFrameTransform
# from data_cleaning_for_EDA import DataFrameInfo 

Read in data: 

In [2]:
def parse_to_datetime_mo_yr(date_str):
    return pd.to_datetime(date_str, format='%b-%Y', errors='coerce') 

# Reading in the csv and parsing dates. 
finance_df = pd.read_csv(
    "dataframe.csv",
    parse_dates=['issue_date', 'earliest_credit_line', 'last_payment_date', 'next_payment_date', 'last_credit_pull_date'],
    date_format='%m-%Y',
    converters={'issue_date': parse_to_datetime_mo_yr, 
                'earliest_credit_line': parse_to_datetime_mo_yr,
                'last_payment_date': parse_to_datetime_mo_yr,
                'next_payment_date': parse_to_datetime_mo_yr,
                'last_credit_pull_date': parse_to_datetime_mo_yr}
)
# Cleaning up lengths of time that could be treated as floats. 
finance_df['employment_length'] = finance_df['employment_length'].str.extract(r"([-+]?\d*\.\d+|[-+]?\d+)").astype(float)
finance_df['term'] = finance_df['term'].str.extract(r"([-+]?\d*\.\d+|[-+]?\d+)").astype(float)
finance_df.rename(columns = {'employment_length':'years_of_employment', 'term' : 'term_length_in_months'}, inplace = True) 
# Cleaning up object type columns- all of which are suitable for category (I checked).
columns_to_cat = ['grade', 'sub_grade', 'verification_status', 'home_ownership', 'loan_status', 'payment_plan', 'purpose', 'application_type']
finance_df[columns_to_cat] = finance_df[columns_to_cat].astype('category')

Allowing my data frame to inherit any classes I've created and want to use: 

In [3]:
class MultipleInheritanceTestClass(im.Plotter, im.DataFrameTransform, dc.DataFrameInfo):
    def export_to_csv(self, filename):
        self.data_frame.to_csv(filename, index=False)
        print(f"DataFrame exported to {filename}")

df = MultipleInheritanceTestClass(finance_df)

Main Workspace: 

In [4]:
# Before: 
df.perc_null()

Unnamed: 0                      0.000000
id                              0.000000
member_id                       0.000000
loan_amount                     0.000000
funded_amount                   5.544799
funded_amount_inv               0.000000
term_length_in_months           8.799395
int_rate                        9.531449
instalment                      0.000000
grade                           0.000000
sub_grade                       0.000000
years_of_employment             3.905515
home_ownership                  0.000000
annual_inc                      0.000000
verification_status             0.000000
issue_date                      0.000000
loan_status                     0.000000
payment_plan                    0.000000
purpose                         0.000000
dti                             0.000000
delinq_2yrs                     0.000000
earliest_credit_line            0.000000
inq_last_6mths                  0.000000
mths_since_last_delinq         57.166565
mths_since_last_

In [5]:
df.drop_weak_columns() 
df.perc_null()


Unnamed: 0                    0.000000
id                            0.000000
member_id                     0.000000
loan_amount                   0.000000
funded_amount                 5.544799
funded_amount_inv             0.000000
term_length_in_months         8.799395
int_rate                      9.531449
instalment                    0.000000
grade                         0.000000
sub_grade                     0.000000
years_of_employment           3.905515
home_ownership                0.000000
annual_inc                    0.000000
verification_status           0.000000
issue_date                    0.000000
loan_status                   0.000000
payment_plan                  0.000000
purpose                       0.000000
dti                           0.000000
delinq_2yrs                   0.000000
earliest_credit_line          0.000000
inq_last_6mths                0.000000
open_accounts                 0.000000
total_accounts                0.000000
out_prncp                

Decide whether the column should be imputed with median or mean: 

In [ ]:
#I ran plots for all five of the non-zero percentage null columns, but I kept pressing shift+enter so moved on with my life; you can see three here and run the other three yourself if you want. 
df.plot_column("int_rate")
df.plot_column("years_of_employment")
df.plot_column("collections_12_mths_ex_med")
#they're all skewed so all median imputes

Impute null values:

In [6]:
df.impute_null_values()
df.perc_null() # All 0s now, as should be. 

Unnamed: 0                    0.0
id                            0.0
member_id                     0.0
loan_amount                   0.0
funded_amount                 0.0
funded_amount_inv             0.0
term_length_in_months         0.0
int_rate                      0.0
instalment                    0.0
grade                         0.0
sub_grade                     0.0
years_of_employment           0.0
home_ownership                0.0
annual_inc                    0.0
verification_status           0.0
issue_date                    0.0
loan_status                   0.0
payment_plan                  0.0
purpose                       0.0
dti                           0.0
delinq_2yrs                   0.0
earliest_credit_line          0.0
inq_last_6mths                0.0
open_accounts                 0.0
total_accounts                0.0
out_prncp                     0.0
out_prncp_inv                 0.0
total_payment                 0.0
total_payment_inv             0.0
total_rec_prnc

In [ ]:
#visualise the removal of null values with another plot: 
df.plot_column("int_rate")
df.plot_column("years_of_employment")
df.plot_column("collections_12_mths_ex_med")
#As expected, now major spikes at point of median-- this would increase kurtosis. 

Task 4: Dealing with Skew. 

In [14]:
#Preamble (speedrun through Task 3 on startup)
# 1. Run imports 
# 2. Read-in finance_df 

class MultipleInheritanceTestClass(im.Plotter, im.DataFrameTransform, dc.DataFrameInfo):
        def export_to_csv(self, file_name):
            self.data_frame.to_csv(file_name, index=False)
            print(f"DataFrame exported to {file_name}")

df = MultipleInheritanceTestClass(finance_df)
df.drop_weak_columns() 
df.impute_null_values() 

Step 1: Identify skewed columns in data using Pandas methods; determine a threshold for skewness. 

I've created a high_skew_columns method in data_clening_for_EDA. It should give any columns with skew greater than 1.2 (default threshold). 

In [ ]:
# finance_df.skew(numeric_only=True)
df.high_skew_columns() # With a threshold of 1.2, this yielded: 
'''
['id',
 'member_id',
 'annual_inc',
 'delinq_2yrs',
 'inq_last_6mths',
 'out_prncp',
 'out_prncp_inv',
 'total_payment',
 'total_payment_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_payment_amount',
 'collections_12_mths_ex_med'] 
'''



Rule of thumb off the internet is a skew of abs val 1 is "uncomfortable." Let's see what those look like. 

In [ ]:
df.plot_column("term_length_in_months") #skew of 1 and this is the "binary" one I imputed with the median so it now has a ratio of about 8:3... pretty heavily skewed! 

In [ ]:
df.plot_column("total_accounts") # skewness of about 0.8 apparently. Looks like a Normal Distribution with just some extra tail on the top end. 

In [ ]:
df.plot_column("total_rec_late_fee") #skewness of 13 - graph basically looks like it has just one outcome -- very skewed. 

Step 2: Once the skewed columns are identified, perform transformations no these columns to determine which transformation results in the biggest reduction in skew. 
Create a method to transform columns in DataFrameTransform class. 

As advised by Mat from AiCore, and detailed further here (https://anatomisebiostats.com/biostatistics-blog/transforming-skewed-data/), I've identified the following may be appropriate transformations: 

- log: potentially appropriate for high, positive skew. the larger the base (e.g. e vs 10), the higher the original degree of skewness. 
    - reciprocal: Mat didn't mention this, but it may be appropriate for extreme positive skewness. 
- yeo-johnson: Potentially appropriate for positively skewed data. 
- box-cox: For positively skewed, exclusively positive data. 
- square root: potentially appropriate for moderate, positive skew. 
- cube root: potentially appropriate for negatively skewed data. 
    - square: potential for negative skew. (again, this was suggested not by Mat but by https://medium.com/@TheDataGyan/day-8-data-transformation-skewness-normalization-and-much-more-4c144d370e55.)

I need to be able visualise these transformations one at a time before I commit them to transforming any given column. Therefore, I'll:
1. Add a transformation attribute to the plot_column() method. 
2. Add a printout of the skew values to the high_skew_columns() method. 

In [ ]:
# Look at all the high-skew plots. 
high_skew_list = df.high_skew_columns()
for column_instance in high_skew_list: 
    print(f"Plot of '{column_instance}' with identity transformation:")
    df.plot_column(column_instance)

From the plots above we can visually confirm that all the highly skewed columns are right skewed, with skewnesses ranging from moderate (1.26) to crazy extreme (27.64). On the basis of the degree of skewness I will try a few of the following transformations, visualising in plotter before commmitting them to data transformation. Since all major skews were positive, I can rule out the transformations better suited to negative skews. All the key data is non-negative but I can't rule out values of zero. I can select the transformation on the basis of strength of original skewness: 
- moderate positive skew: square root
- high positive skew: natural log 
- higher positive skew: log base 10 
- even higher positive skew: log base 100
- extreme positive skew: reciprocal 

In [8]:
sqrt_transform = lambda x: pd.Series(x).apply(lambda val: val if val < 0 else np.sqrt(val)) #negatives shouldn't be an issue, but just in case... 
ln_transform = lambda x: pd.Series(x).apply(lambda val: val if val <= 0 else np.log(val)) # log 0 doesn't work either. 
log10_transform = lambda x: pd.Series(x).apply(lambda val: val if val <= 0 else np.log10(val))
log100_transform = lambda x: pd.Series(x).apply(lambda val: val if val <= 0 else np.log(val)/np.log(100)) #numpy doesn't default to log base 100, so need to use log rules. 
recip_transform = lambda x: pd.Series(x).apply(lambda val: val if val == 0 else np.reciprocal(val)) # reciprocal always exists, except at 0


Now I'll apply those tranformations and evaluate their prospects according to the list in finance_df.skew().txt (soon to appear in this repo).

Starting with the skewiest: 

In [ ]:
df.plot_column(column_name="collection_recovery_fee", transformer = recip_transform)

The reciprocal made almost no dent on collection_recovery_fee! Let's try log100: 

In [ ]:
df.plot_column(column_name="collection_recovery_fee", transformer = log100_transform) # that looks better 

Next up I'll try the reciprocal and log100 on total_rec_late_fee (skew =13), recoveries (skew = 14), and collections_12_mths_ex_med (skew 20). 

In [ ]:
high_3_skews = {"total_rec_late_fee", "recoveries", "collections_12_mths_ex_med"}
for column_instance in high_3_skews:
    print(f"Plot of '{column_instance}' with reciprocal transformation:")
    df.plot_column(column_instance, transformer= recip_transform)
    print(f"Plot of '{column_instance}' with log100 transformation:")
    df.plot_column(column_instance, transformer= log100_transform)

Selections have been made for the above, see .txt file. 
Now onto the high, but not as high skews: annual_inc (skew = 8.7), deling_2yrs (5.4), inq_last_6mths (3.2)


In [ ]:
mid_high_skews = {"annual_inc", "delinq_2yrs", "inq_last_6mths"}
for column_instance in mid_high_skews:
    print(f"Plot of '{column_instance}' with log10 transformation:")
    df.plot_column(column_instance, transformer= log10_transform)
    print(f"Plot of '{column_instance}' with log100 transformation:")
    df.plot_column(column_instance, transformer= log100_transform)

It was after the above plots that I noticed the log transformations are identical barring their scale, which should not effect skewness. Thus, whether I apply log10 or log100 (or ln) should make no difference to the skewness of the transformed data. An alternative transformation I could have tried was the yeo-johnson transformation but I'm running low on time so I skipped it for now, but I recognise I could have used it (from https://towardsdatascience.com/types-of-transformations-for-better-normal-distribution-61c22668d3b9 : 

scipy.stats import yeojohnson
yf_target, lam = yeojohnson(df["TARGET"]) 

). 

Moving on... 
Then there were loads in the 2-3 range: last_payment_amount, total_rec_int, out_prncp_inv, out_prncp, member_id, id (these last two aren't relevant, as they are ids, not measures of anything in particular). Bear in mind that the dual displays for log10 and ln below should be identical for each column, if my understanding is correct. 

In [ ]:
mid_skews = {"last_payment_amount", "total_rec_int", "out_prncp_inv", "out_prncp"}
for column_instance in mid_skews:
    print(f"Plot of '{column_instance}' with log10 transformation:")
    df.plot_column(column_instance, transformer= log10_transform)
    print(f"Plot of '{column_instance}' with ln transformation:")
    df.plot_column(column_instance, transformer= ln_transform)


Though the graphs look identical across relative scales, the internet assures me that skew is affected differently by different sizes of log base. 

Finally there were the skews greater than 1.2 but less than 2: total_payment, total_payment_inv, total_rec_prncp 

In [ ]:
moderate_skews = {"total_payment","total_payment_inv", "total_rec_prncp"}
for column_instance in moderate_skews:
    print(f"Plot of '{column_instance}' with sqrt transformation:")
    df.plot_column(column_instance, transformer= sqrt_transform)
    print(f"Plot of '{column_instance}' with ln transformation:")
    df.plot_column(column_instance, transformer= ln_transform)

Try transforming all those columns: 

In [10]:
#the following selected from finance_df.skew().txt: 
columns_to_fix = df.high_skew_columns()[2:] # drop the id numbers 
# print(columns_to_fix) # confirmed the above did what i wanted 
transforms_to_apply = [log10_transform, log10_transform, log10_transform, log10_transform, log10_transform, sqrt_transform, sqrt_transform, sqrt_transform, ln_transform, log100_transform, log100_transform, log100_transform, log10_transform, recip_transform]

# I have confirmed that both of the above are lists. 

# print(len(columns_to_fix), len(transforms_to_apply)) # of the same length: 14 and 14

df.skew_transform(columns_to_fix, transforms_to_apply)

Transformation applied to column 'annual_inc'.
Transformation applied to column 'delinq_2yrs'.
Transformation applied to column 'inq_last_6mths'.
Transformation applied to column 'out_prncp'.
Transformation applied to column 'out_prncp_inv'.
Transformation applied to column 'total_payment'.
Transformation applied to column 'total_payment_inv'.
Transformation applied to column 'total_rec_prncp'.
Transformation applied to column 'total_rec_int'.
Transformation applied to column 'total_rec_late_fee'.
Transformation applied to column 'recoveries'.
Transformation applied to column 'collection_recovery_fee'.
Transformation applied to column 'last_payment_amount'.
Transformation applied to column 'collections_12_mths_ex_med'.


Now I will save a copy of the df dataframe so I can work with it further without running all that ^ code next time... 

In [15]:
# print(type(df)) # confirms I can't use a pd method directly 
# so I added an export method to the MultipleInheritanceTestClass() 

df.export_to_csv("df_end_of_Task_4")

DataFrame exported to df_end_of_Task_4
